In [1]:
import os, string, numpy, pandas, nltk

docnames = os.listdir('transcripts')
doccount = len(docnames) #404 vlogs

dtdf = pandas.DataFrame(index = docnames, columns = ['#Tokens', 'Tokens']) #DOCUMENT DF: rows = doc names, cols = num tokens & string of all tokens

alltxt =""
for vlogi in (docnames):
    vltxt =  open('transcripts/' + vlogi, encoding='utf-8-sig').read()
    vltxt = vltxt.lower().replace('\ufeff', '')
    alltxt += vltxt
    doctokens = nltk.word_tokenize(vltxt.translate(str.maketrans('','',string.punctuation)))
    dtdf.loc[vlogi] = pandas.Series({'#Tokens':len(doctokens), 'Tokens':doctokens}) # fill data

alltokens = nltk.word_tokenize(alltxt.translate(str.maketrans('','',string.punctuation))) #list of all tokens in the db including repeats
uniqs = set(alltokens)

alltkct = {} #for each token, what is the count for that token in the entire db
for tkn in alltokens:
    try: alltkct[tkn] += 1
    except KeyError: alltkct[tkn] = 1

onces = 0
for tkn in alltkct:
    if(alltkct[tkn] == 1):
        onces += 1
        
tstatdf = pandas.DataFrame.from_dict(alltkct, orient='index').rename(columns={0:'TF'}) #TOKEN DF: rows = tokens, cols = statistics
tstatdf.sort_values(['TF'], ascending= False, inplace = True)
tstatdf = tstatdf.iloc[:30]

tstatdf['N'] = doccount
tstatdf['DF'] = 0

for tkn in tstatdf.index.values:
    for tokens in dtdf['Tokens']:
        if tkn in tokens:
            tstatdf.loc[tkn,'DF'] += 1

tstatdf['IDF'] = numpy.log(tstatdf['N'].divide(tstatdf['DF']))
tstatdf['TFxIDF'] = tstatdf['TF'] * tstatdf['IDF']
tstatdf['Prob'] = tstatdf['TF'].divide(len(alltokens))

print('1. There are', len(alltokens), 'word tokens in the database.')
print('2. From these total tokens, there are', len(uniqs), 'unique tokens.')
print('3.', onces, 'tokens were said only once.')
print('4. Statistics for the 30 most frequent words are shown below.')
print(tstatdf)
print('5. There are', len(alltokens)/404, 'tokens per document on average.')


1. There are 237224 word tokens in the database.
2. From these total tokens, there are 10686 unique tokens.
3. 4614 tokens were said only once.
4. Statistics for the 30 most frequent words are shown below.
          TF    N   DF       IDF      TFxIDF      Prob
i       9867  404  401  0.007453   73.543198  0.041594
and     7958  404  401  0.007453   59.314560  0.033546
the     6490  404  398  0.014963   97.109044  0.027358
to      6253  404  403  0.002478   15.496910  0.026359
a       5315  404  399  0.012453   66.190146  0.022405
you     5111  404  395  0.022529  115.146297  0.021545
that    4112  404  386  0.045578  187.414715  0.017334
it      3813  404  387  0.042990  163.921575  0.016073
of      3626  404  389  0.037836  137.191648  0.015285
so      3142  404  389  0.037836  118.879249  0.013245
like    3052  404  363  0.107012  326.600757  0.012865
is      2510  404  381  0.058616  147.124912  0.010581
my      2490  404  371  0.085213  212.179910  0.010496
in      2456  404  381  